##  社交网络分析数据处理

In [64]:
def time_stats(dfGroupData):
    dfGroupData = dfGroupData.agg( countDistinct("custorm_id").alias("cust_num"),
                                            count("time").alias("freq_num"),
                                            sum("time").alias("sum_time"),  ## 时间的总计
                                            avg("time").alias("avg_time"),
                                            max("time").alias("longest_time"),
                                            min("time").alias("shortest_time"),
                                            var_samp("time").alias("var_time"),
                                            stddev_pop("time").alias("std_time"),
                                            kurtosis("time").alias("kurt_time"),


                                            sum("call_time_lag").alias("sum_call_time_lag"),  ##通话频率的的总计
                                            avg("call_time_lag").alias("avg_call_time_lag"),
                                            max("call_time_lag").alias("longest_call_time_lag"),
                                            min("call_time_lag").alias("shortest_call_time_lag"),
                                            var_samp("call_time_lag").alias("var_call_time_lag"),
                                            stddev_pop("call_time_lag").alias("std_call_time_lag"),
                                            kurtosis("call_time_lag").alias("kurt_call_time_lag"),

                                            # 1点到7点打电话的次数[01,07]闭区间 ，下面依次对总时间，平均时间，最长，最短，标准差等进行统计
                                            count(when(time_hour_cond(1, 8), "time")).alias("call_time_cnt_0107"),
                                            count(when(time_hour_cond(8, 13), "time")).alias("call_time_cnt_0812"),
                                            count(when(time_hour_cond(13, 18), "time")).alias("call_time_cnt_1317"),
                                            count(when(time_hour_cond(18, 22), "time")).alias("call_time_cnt_1821"),
                                            count(when(time_hour_cond(22, 24) | (call_record["call_time_hour"] == 0),
                                                       "time")).alias("call_time_cnt_2200"),

                                            sum(when(time_hour_cond(1, 8), call_record["time"])).alias(
                                                "call_time_sum_0107"),
                                            sum(when(time_hour_cond(8, 13), call_record["time"])).alias(
                                                "call_time_sum_0812"),
                                            sum(when(time_hour_cond(13, 18), call_record["time"])).alias(
                                                "call_time_sum_1317"),
                                            sum(when(time_hour_cond(18, 22), call_record["time"])).alias(
                                                "call_time_sum_1821"),
                                            sum(when(time_hour_cond(22, 24) | (call_record["call_time_hour"] == 0),
                                                     call_record["time"])).alias("call_time_sum_2200"),

                                            avg(when(time_hour_cond(1, 8), call_record["time"])).alias(
                                                "call_time_avg_0107"),
                                            avg(when(time_hour_cond(8, 13), call_record["time"])).alias(
                                                "call_time_avg_0812"),
                                            avg(when(time_hour_cond(13, 18), call_record["time"])).alias(
                                                "call_time_avg_1317"),
                                            avg(when(time_hour_cond(18, 22), call_record["time"])).alias(
                                                "call_time_avg_1821"),
                                            avg(when(time_hour_cond(22, 24) | (call_record["call_time_hour"] == 0),
                                                     call_record["time"])).alias("call_time_avg_2200"),

                                            stddev_pop(when(time_hour_cond(1, 8), call_record["time"])).alias(
                                                "call_time_stddev_pop_0107"),
                                            stddev_pop(when(time_hour_cond(8, 13), call_record["time"])).alias(
                                                "call_time_stddev_pop_0812"),
                                            stddev_pop(when(time_hour_cond(13, 18), call_record["time"])).alias(
                                                "call_time_stddev_pop_1317"),
                                            stddev_pop(when(time_hour_cond(18, 22), call_record["time"])).alias(
                                                "call_time_stddev_pop_1821"),
                                            stddev_pop(
                                                when(time_hour_cond(22, 24) | (call_record["call_time_hour"] == 0),
                                                     call_record["time"])).alias("call_time_stddev_pop_2200"),

                                            max(when(time_hour_cond(1, 8), call_record["time"])).alias(
                                                "call_time_max_0107"),
                                            max(when(time_hour_cond(8, 13), call_record["time"])).alias(
                                                "call_time_max_0812"),
                                            max(when(time_hour_cond(13, 18), call_record["time"])).alias(
                                                "call_time_max_1317"),
                                            max(when(time_hour_cond(18, 22), call_record["time"])).alias(
                                                "call_time_max_1821"),
                                            max(when(time_hour_cond(22, 24) | (call_record["call_time_hour"] == 0),
                                                     call_record["time"])).alias("call_time_max_2200"),
                                            )
    dfGroupData = dfGroupData.withColumn("avg_everyday_call_time",dfGroupData['sum_time'] / dfGroupData["sum_call_time_lag"]).withColumn( \
        "std_time_lag", dfGroupData['std_time'] / dfGroupData["std_call_time_lag"]).na.fill(0.0)
    return dfGroupData

In [76]:
def Group_Major(major_key,call_record):
    #--------------------------窗口函数-----------------------------------------------------
    #重新分区并对分区进行排序，方便相减
    if isinstance(major_key, list):
        call_record=call_record.repartition(major_key[0],major_key[1]).sortWithinPartitions("call_time",ascending=False)
    ## 建立窗口，那所有的操作都是在窗口中进行操作的，相当于impala over 函数 lag(call_time) over (partition by name ,number order by call_time desc)

        window=Window.partitionBy(major_key[0],major_key[1])
    else:
        call_record=call_record.repartition(major_key).sortWithinPartitions("call_time",ascending=False)

        window=Window.partitionBy(major_key)

    #时间上的相减，本文并没有采用datediff 主要是想要才用比较精确的时间，所以转换成时间戳直接相减，这也是比较好的时间处理方式
    call_record = call_record.withColumn("call_time_lag", (unix_timestamp(lag("call_time").over(window)) - unix_timestamp(call_record["call_time"])) / 86400)
    ## hour,month,minute 是去'2016-10-22 00:11:52'中的某一个时间，spark 的DataFrame 的操作很大一部分是对hive的一种实现，只是更加的方便可以用Python等语言自定义函数
    call_record_group=call_record.groupBy(major_key)
    # 分组[nma,nymber]为主键进行聚合



    call_record_agg=time_stats(call_record_group)

    return call_record_agg

def agg_func(names, func):
    A = dict()
    for name in names:
        A[name] = func
    return A

def add_suffix(df,suffix,start=1):
    for col_name in  df.columns[start:]:
        df=df.withColumnRenamed(col_name,col_name+suffix)
    return df

In [84]:
data=sqlContext.read.parquet("crawler_mobi_call_record_hive.pqt")
data.registerTempTable("mobi_call")
call_record=sqlContext.sql("select * from  mobi_call where length(login_name)>9 and num_contact<2700 and num_contact !=0")

In [85]:
from pyspark.sql import DataFrame,HiveContext
from pyspark.sql.functions import  *
from pyspark.sql import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types  import DoubleType
# call_record=sqlContext.read.parquet("mobi_call_record_sample.pqt")
call_record=call_record.withColumnRenamed("time_contact","call_time").withColumnRenamed("Phone_contact",
            "number").withColumn("time",call_record.num_contact.cast("float")).withColumn("type",call_record.type_call.cast("float"))
call_record = call_record.withColumn("call_time_hour", hour("call_time").cast(DoubleType())).drop("num_contact").drop("type_call")
time_hour_cond = lambda start, end: (call_record["call_time_hour"] >= start) & (call_record["call_time_hour"] < end)
type_cond = lambda flag: call_record['type'] == flag


In [71]:
sam=call_record.toPandas().head(10)
sam

,custorm_id,login_name,call_time,number,time,type,call_time_hour
0,1000021,13759806199,2016-01-17 18:05:15,13038458880,35,1,18
1,1000021,13759806199,2016-01-18 21:00:11,13571612672,73,1,21
2,1000021,13759806199,2016-02-03 14:33:07,18329462784,35,1,14
3,1000021,13759806199,2016-02-16 16:47:13,13289609216,24,1,16
4,1000021,13759806199,2016-02-17 14:47:05,13891689472,55,2,14
5,1000021,13759806199,2016-03-24 09:21:28,13891680256,19,2,9
6,1000021,13759806199,2016-04-07 08:46:53,13891680256,16,2,8
7,1000021,13759806199,2016-04-18 13:08:35,18700642304,59,1,13
8,1000021,13759806199,2016-04-21 13:05:32,13891671040,5,1,13
9,1000021,13759806199,2016-05-10 09:53:55,18086014976,19,2,9


In [72]:
sam.type

0    1
1    1
2    1
3    1
4    2
5    2
6    2
7    1
8    1
9    2
Name: type, dtype: float64

In [87]:
call_record.cache()
print call_record.count()
## step1:以number为主键
major_key="number"
call_record_number_agg=Group_Major(major_key,call_record)
call_record_number_agg.saveAsParquetFile("mobi_call_record_number_agg.pqt")

1959215405


In [1]:
sample=sqlContext.read.parquet("mobi_call_record_number_agg.pqt")
sample.count()

2322795

In [7]:
sample.registerAsTable("sample")
# cc=sqlContext.sql("select  * from  sample  limit 100")


In [11]:
cc=sample.head(100)
col_names=row_name(cc[0])
col_names

['call_time_sum_0812',
 'call_time_cnt_1821',
 'number',
 'var_call_time_lag',
 'kurt_call_time_lag',
 'call_time_stddev_pop_0107',
 'call_time_cnt_0812',
 'avg_call_time_lag',
 'call_time_avg_2200',
 'longest_call_time_lag',
 'avg_everyday_call_time',
 'avg_time',
 'call_time_cnt_1317',
 'freq_num',
 'std_time',
 'call_time_max_1317',
 'shortest_time',
 'call_time_max_1821',
 'call_time_sum_2200',
 'sum_time',
 'shortest_call_time_lag',
 'longest_time',
 'call_time_sum_0107',
 'call_time_sum_1821',
 'call_time_max_0812',
 'std_time_lag',
 'var_time',
 'call_time_avg_1821',
 'std_call_time_lag',
 'call_time_stddev_pop_1317',
 'call_time_avg_0107',
 'call_time_avg_1317',
 'sum_call_time_lag',
 'kurt_time',
 'call_time_stddev_pop_2200',
 'call_time_stddev_pop_1821',
 'call_time_max_0107',
 'call_time_avg_0812',
 'cust_num',
 'call_time_stddev_pop_0812',
 'call_time_cnt_2200',
 'call_time_max_2200',
 'call_time_sum_1317',
 'call_time_cnt_0107']

In [4]:
# sample_df=sample.toPandas()
# sample_df.to_csv("mobi_call_record.csv")
row_list=lambda row:row.asDict(True).values()
row_name=lambda row:row.asDict(True).keys()
sample.rdd.map(row_list).repartition(1).saveAsTextFile("mobi_call_record_hive1.csv")

In [29]:
import pandas as pd
import numpy as np
mobi_call=pd.read_csv("scoial_network/mobi_call_record_hive.csv",header=col_names)

TypeError: cannot concatenate 'str' and 'int' objects

In [34]:
mobi=open("scoial_network/mobi_call_record_hive.csv")
mobi=mobi.readlines()

In [35]:
def convent_list(line_str):
    dd=line_str.strip()[1:-1].split(",")
    return [float(line.strip())  for line in dd]

In [36]:
mobi=np.array([convent_list(line)  for line in mobi])

In [37]:
mobi_df=pd.DataFrame(mobi,columns=col_names)

In [40]:
mobi_df.set_index("number",inplace=True)

In [41]:
mobi_df.head(3)

,call_time_sum_0812,call_time_cnt_1821,var_call_time_lag,kurt_call_time_lag,call_time_stddev_pop_0107,call_time_cnt_0812,avg_call_time_lag,call_time_avg_2200,longest_call_time_lag,avg_everyday_call_time,...,call_time_stddev_pop_2200,call_time_stddev_pop_1821,call_time_max_0107,call_time_avg_0812,cust_num,call_time_stddev_pop_0812,call_time_cnt_2200,call_time_max_2200,call_time_sum_1317,call_time_cnt_0107
number,,,,,,,,,,,,,,,,,,,,,
13001517056,2734,25,11791.564687,0.095679,13.242356,50,-0.189845,30.000000,297.461042,-282.367651,...,27.294688,59.571053,43,54.680000,54,61.852547,4,77,4296,5
13002131456,4243,29,6286.867394,0.481594,0.000000,33,2.054385,84.333333,200.509641,69.096683,...,127.412279,200.117367,105,128.575758,41,203.375907,6,368,8312,1
13003360256,11038,115,11035.752957,0.451520,307.395816,184,0.726211,99.200000,352.170602,100.850493,...,226.590732,125.672733,1166,59.989130,75,85.480038,10,778,13996,12


In [43]:
mobi_df.to_csv("scoial_network/crawler_mobi_call_record_hive.csv")